# Análise de Transações PIX
Metodologia: CRISP-DM - https://www.escoladnc.com.br/blog/data-science/metodologia-crisp-dm/
### Objetivos
- Limpar e pré-processar os dados das transações PIX
- Analisar padrões de uso do PIX, tais como os canais mais utilizados e os valores de transação mais comuns
- Usar o PySpark MLlib para treinar e avaliar um modelo de detecção de fraude
- Avaliar o desempenho do modelo e fazer recomendações para melhorias futuras

### Dados

O conjunto de dados inclui as seguintes informações para cada transação:
- Detalhes da transação: valor, tempo, remetente e receptor CPF/CNPJ, tipo
- Etiqueta de fraude: uma variável binária que indica se a transação foi fraudulenta (1) ou não (0)

### Tarefas
- Normalização dos dados:
  - O dataset que você lerá está em formato json.
  ```json
  {
      "id_transacao": inteiro,
      "valor": texto,
      "remetente": {
          "nome": texto,
          "banco": texto,
          "tipo": texto
      },
      "destinatario": {
          "nome": texto,
          "banco":texto,
          "tipo": texto
      },          
      "categoria":texto,
      "chave_pix":texto,
      "transaction_date":texto,
      "fraude":inteiro,
  }
    ```
  - Será necessário fazer a transformação para formato colunar
- Análise Exploratória de Dados: Use o PySpark para analisar padrões de uso do PIX:
  - chaves pix mais usadas;
  - os valores de transação mais comuns;
  - distribuição dos valores de transação por hora e dia;
  - quais bancos receberam mais transferências por dia;
  - para qual tipo de pessoa (PF ou PJ) foram realizadas mais transações
- Engenharia de Recursos: Apresentar novas características que podem ser úteis para a detecção de fraudes, tais como o número de transações feitas pelo mesmo remetente em um período de tempo específico.
- Modelagem: Use o PySpark MLlib para treinar e detectar possíveis transações que contenham fraude.

### Observação
É importante notar que este é um caso simplificado, e em cenários do mundo real você teria que lidar com dados mais complexos, usar técnicas mais avançadas como métodos de conjuntos e considerar o conhecimento de domínio, bem como leis e regulamentos das instituições financeiras no Brasil.

# Entendimento do Negócio
Através da base de transações do pix o banco deseja entender qual é o perfil dos clientes que utilizam o pix, além de verificar possíveis transações que tenham fraude. Porém, eles tem um cliente específico que tem um relacionamento muito bom para o banco, por isso, você recebeu a base de transações de cliente dos últimos 2 anos e precisa a partir dela criar um relatório contendo as principais características das transações.


Então, resumindo, temos dois principais objetivos para esse case:
1. Obter valor a partir dos dados
  - Para qual banco esse cliente mais transfere?
  - Qual é a média de transferências por período que esse cliente faz?
  - Baseando-se no valor das transferências, poderia dar um aumento de crédito?
  - Para o que esse cliente mais usa as transferências?
2. Executar um algoritmo de machine learning que identifique possíveis transações com fraude.
3. Pós Processamento
  - Defina ao mínimo cinco métricas de qualidade para seus dados
  - Explique se os seus dados estão com uma boa qualidade

# Preparação do Ambiente de Desenvolvimento

In [2]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3e68c4a986ccfd4358be964d827903d283d83049e48b1d8dad991c623fa7c394
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Data Undesrtanting

Primeiramente, devemos entender tudo sobre a fonte dos dados
- Como o dado chega até nós?
- Qual formato virá?
- Aonde o processamento será executado (AWS EMR, Cluster On-Premise)?
- De quanto em quanto tempo eu preciso gerar esse relatório (mensal, diário, near-real time)?

```json
{
  "id_transacao": inteiro,
  "valor": texto,
  "remetente": {
      "nome": texto,
      "banco": texto,
      "tipo": texto
  },
  "destinatario": {
      "nome": texto,
      "banco":texto,
      "tipo": texto
  },
  "categoria": texto,
  "transaction_date":texto,
  "chave_pix":texto,
  "fraude":inteiro,
}
```



# Preparação dos Dados
Agora é hora de começar a preparar os dados de acordo com as suas necessidades.

In [91]:
# Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import month, year, to_date
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import floor

In [3]:
# Iniciar a sessão spark
spark = (
  SparkSession.builder
    .config('spark.ui.port', '4050')
    .appName('AnalisePix')
    .getOrCreate()
)

In [5]:
caminho_json = '/content/case_final.json'

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

In [65]:
df = spark.read.json(
    caminho_json,
    schema = schema_base_pix,
    timestampFormat = "yyyy-MM-dd HH:mm:ss"
)

In [66]:
df.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)



In [68]:
df.show()

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588.08|{Jonathan Gonsalv...|{Calebe Melo, Cai...|aleatoria|       outros|2021-07-16 05:00:55|     0|
|        1001|           80682.5|{Jonathan Gonsalv...|{Davi Lucas Perei...|  celular|transferencia|2022-04-20 12:34:01|     1|
|        1002|             549.9|{Jonathan Gonsalv...|{Sabrina Castro, ...|      cpf|        lazer|2022-07-10 16:51:34|     0|
|        1003|             90.83|{Jonathan Gonsalv...|{Francisco da Con...|aleatoria|   transporte|2022-10-20 10:57:36|     0|
|        1004|13272.619999999999|{Jonathan Gonsalv...|{Isabelly Ferreir...|    email|transferencia|2021-04-06 2

In [69]:
# Criando o DataFrame Flatten

df_flatten = df.withColumns({
    'remetente_nome': col('remetente').getField('nome'),
    'remetente_banco':  col('remetente').getField('banco'),
    'remetente_tipo': col('remetente').getField('tipo'),
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo')
}).drop('remetente', 'destinatario')

In [70]:
df_flatten.show()

+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|
+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|            BTG|            PF|         Calebe Melo|             Caixa|               PF|
|        1001|           80682.5|  celular|transferencia|2022-04-20 12:34:01|     1|Jonathan Gonsalves|            BTG|            PF|  Davi Lucas Pereira|             Caixa|               PJ|
|        1002|             549.9|  

In [20]:
df_flatten.describe().show()

+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|summary|     id_transacao|             valor|chave_pix|  categoria|   transaction_date|            fraude|    remetente_nome|remetente_banco|remetente_tipo|destinatario_nome|destinatario_banco|destinatario_tipo|
+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|  count|           100000|            100000|   100000|     100000|             100000|            100000|            100000|         100000|        100000|           100000|            100000|           100000|
|   mean|          50999.5|10303.358732200059|     NULL|       NULL|               NULL|           0.15367|              NULL|           NULL|      

# Modelagem
Aqui você encontrará utilidade para os dados levantados.

Aqui será onde teremos insights e, a partir desses, novos conhecimentos sobre o business (se tudo até aqui foi feito corretamente).


- Para qual banco esse cliente mais transfere?
- Qual é a média de transferências por período que esse cliente faz?
- Baseando-se no valor das transferências, poderia dar um aumento de crédito?
- Para o que esse cliente mais usa as transferências?
- Executar um algoritmo de machine learning que identifique possíveis transações com fraude.


In [23]:
# Bancos que o cliente mais transfere
df_flatten.groupby('destinatario_banco').count().orderBy('count').show()

+------------------+-----+
|destinatario_banco|count|
+------------------+-----+
|          Bradesco|14187|
|                C6|14204|
|             Caixa|14240|
|              Itau|14281|
|            Nubank|14297|
|               BTG|14390|
|                XP|14401|
+------------------+-----+



In [ ]:
df_flatten.groupby(
    date_format("transaction_date", "yyyy-MM").alias('ano_mes'),
    'destinatario_banco'
).count().orderBy(col('ano_mes').desc()).show()

In [46]:
# Média de transferências por período
df_flatten.groupby(
    'destinatario_banco'
).avg('valor').orderBy('avg(valor)').show()


+------------------+------------------+
|destinatario_banco|        avg(valor)|
+------------------+------------------+
|               BTG|10122.299803335622|
|              Itau|10230.876305580874|
|             Caixa|10254.864015449395|
|                C6|10309.499774711307|
|            Nubank|10316.475401133126|
|                XP|10328.071572113045|
|          Bradesco| 10564.19458870794|
+------------------+------------------+



In [48]:
# Média de transferências por período
df_flatten.groupby(
    date_format("transaction_date", "yyyy-MM").alias('ano_mes'),
    'destinatario_banco'
).avg('valor').orderBy(col('avg(valor)').desc()).show()


+-------+------------------+------------------+
|ano_mes|destinatario_banco|        avg(valor)|
+-------+------------------+------------------+
|2022-01|          Bradesco|12820.046042692942|
|2022-08|               BTG|12564.292077702705|
|2022-04|                XP|12263.473821548825|
|2022-10|              Itau| 12217.11777969019|
|2021-08|          Bradesco|12158.968752052559|
|2022-07|                C6|12062.153634812286|
|2022-08|              Itau|11801.010616883113|
|2022-08|             Caixa|11733.853470394733|
|2021-12|                C6|11687.408390243903|
|2021-10|          Bradesco|11643.334805825241|
|2023-01|             Caixa|11642.769566787007|
|2022-01|               BTG|11638.974383775358|
|2021-10|                C6|11603.620265339967|
|2022-05|          Bradesco|11524.133602584816|
|2021-09|          Bradesco| 11446.30193661971|
|2021-08|               BTG|11441.186908212561|
|2022-03|            Nubank| 11435.54794567062|
|2021-12|          Bradesco|11400.090463

In [49]:
# Uso das transferências
df_flatten.groupby(
    date_format("transaction_date", "yyyy-MM").alias('ano_mes'),
    'destinatario_banco',
    'categoria'
).count().orderBy('count').show()

+-------+------------------+-----------+-----+
|ano_mes|destinatario_banco|  categoria|count|
+-------+------------------+-----------+-----+
|2023-01|              Itau| transporte|   15|
|2023-01|              Itau|  presentes|   15|
|2023-01|              Itau|  vestuario|   19|
|2023-01|          Bradesco|  presentes|   20|
|2023-01|             Caixa|     outros|   20|
|2021-01|          Bradesco|  presentes|   21|
|2023-01|             Caixa|      lazer|   21|
|2021-01|                C6| transporte|   21|
|2023-01|            Nubank|   educacao|   21|
|2023-01|                XP|alimentacao|   21|
|2021-01|              Itau|  presentes|   21|
|2023-01|                C6|  presentes|   21|
|2023-01|             Caixa|  vestuario|   21|
|2023-01|                XP|      saude|   21|
|2023-01|            Nubank|alimentacao|   22|
|2023-01|          Bradesco|     outros|   22|
|2021-01|               BTG|     outros|   22|
|2023-01|               BTG|  presentes|   22|
|2023-01|    

In [54]:
# Uso das transferências
df_flatten.groupby(
    date_format("transaction_date", "yyyy").alias('ano'),
    'categoria'
).sum('valor').select('ano', 'categoria', col('sum(valor)').cast(DecimalType(38,3)).alias('valor')).orderBy(col('valor').desc()).show()

+----+-------------+-------------+
| ano|    categoria|        valor|
+----+-------------+-------------+
|2022|transferencia|430191098.790|
|2021|transferencia|416670741.700|
|2023|transferencia| 16148682.620|
|2022|    vestuario| 10696006.480|
|2022|       outros| 10566645.100|
|2022|   transporte| 10553408.290|
|2022|  alimentacao| 10545783.230|
|2021|    presentes| 10496042.020|
|2021|    vestuario| 10412651.930|
|2021|        saude| 10384662.820|
|2022|     educacao| 10367124.440|
|2022|    presentes| 10311585.740|
|2022|        lazer| 10295747.630|
|2021|  alimentacao| 10237928.250|
|2021|     educacao| 10106095.070|
|2022|        saude| 10048245.070|
|2021|       outros|  9737076.380|
|2021|        lazer|  9622503.140|
|2021|   transporte|  9497893.470|
|2023|       outros|   501308.340|
+----+-------------+-------------+
only showing top 20 rows



In [55]:
# Categoria que possui fraude
df_flatten.groupby('categoria', 'fraude').count().show()

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|       outros|     0| 9377|
|     educacao|     0| 9460|
|transferencia|     0| 9377|
|transferencia|     1|15367|
|    presentes|     0| 9254|
|        saude|     0| 9476|
|        lazer|     0| 9464|
|   transporte|     0| 9174|
|    vestuario|     0| 9503|
|  alimentacao|     0| 9548|
+-------------+------+-----+



In [71]:
df_flatten.filter(
    col('categoria') == 'transferencia'
).groupby('categoria', 'fraude').count().show()

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|transferencia|     0| 9377|
|transferencia|     1|15367|
+-------------+------+-----+



In [72]:
# Coluna com Ranges de valores de fraude
df_flatten.filter(col('fraude') == 1).withColumn(
    'range',
    floor(col("valor")/1000)*1000
).groupBy('range').count().orderBy(col('range').desc()).show()

+-----+-----+
|range|count|
+-----+-----+
|89000|  222|
|88000|  208|
|87000|  230|
|86000|  203|
|85000|  205|
|84000|  245|
|83000|  206|
|82000|  206|
|81000|  214|
|80000|  213|
|79000|  205|
|78000|  230|
|77000|  237|
|76000|  232|
|75000|  190|
|74000|  207|
|73000|  237|
|72000|  234|
|71000|  234|
|70000|  222|
+-----+-----+
only showing top 20 rows



In [74]:
df_flatten.filter(col('fraude') == 1).withColumn(
    'range',
    floor(col("valor")/1000)*1000
).select(max('range'), min('range')).show()

+----------+----------+
|max(range)|min(range)|
+----------+----------+
|     89000|     19000|
+----------+----------+



## Modelo de Predição de Fraudes

In [76]:
df = df_flatten.drop('remetente', 'id_transacao')

In [78]:
indexer = StringIndexer(
    inputCols=[
        "destinatario_nome",
        "destinatario_banco",
        "destinatario_tipo",
        "categoria",
        "chave_pix"
              ],
    outputCols=[
        "destinatario_nome_index",
        "destinatario_banco_index",
        "destinatario_tipo_index",
        "categoria_index",
        "chave_pix_index"
        ])

In [79]:
df_index = indexer.fit(df).transform(df)
df_index.show()

+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|            BTG|            PF|         Calebe Melo|   

In [80]:
cols_para_filtrar = [
    "valor",
    "transaction_date",
    "destinatario_nome_index",
    "destinatario_banco_index",
    "destinatario_tipo_index",
    "categoria_index",
    "chave_pix_index",
    "fraude"
]

In [81]:
is_fraud = df_index.select(cols_para_filtrar).filter(col('fraude') == 1)
not_fraud = df_index.select(cols_para_filtrar).filter(col('fraude') == 0)

# Separando uma amostra
not_fraud = not_fraud.sample(False,fraction=0.1, seed=123)

In [83]:
df_concat = not_fraud.union(is_fraud)
df = df_concat.sort('transaction_date')
df.count()

23798

In [84]:
train, test = df.randomSplit([0.7, 0.3], seed=123)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 16567
Test Dataset Count: 7231


In [86]:
is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [89]:
# Criando feature vetores
# VectorAssembler combina listas de colunas em um vetor simples
assembler = VectorAssembler(
    inputCols= [x for x in train.columns if x not in ["transaction_date", "fraude", "is_fraud"]],
    outputCol = 'features')

# Regressão Logistica
lr = LogisticRegression().setParams(
    maxIter= 100000,
    labelCol = 'is_fraud',
    predictionCol= 'prediction')

# Define a pipeline do modelo
model = Pipeline(stages=[assembler, lr]).fit(train)

In [90]:
predicted = model.transform(test)

# Resultado do modelo
predicted = predicted.withColumn("is_fraud", is_fraud(predicted.fraude))
predicted.crosstab("is_fraud", "prediction").show()

+-------------------+----+----+
|is_fraud_prediction| 0.0| 1.0|
+-------------------+----+----+
|                1.0|   0|4636|
|                0.0|2594|   1|
+-------------------+----+----+



# Avaliação do Modelo
Será que seu modelo atinge todas as necessidades que foram definidas inicialmente? (e.g. pessoa em cima da bicicleta muda o resultado final)



# Deployment
Apresente o relatório com os resultados obtidos.

Foi identificado que o cliente Jonathan Gonsalves tem uma alta taxa de transaferências pix mensal. Através de análise realizadas foi possível perceber que a maior categoria de transação é a transferência bancária.

Também foi possível notar que o segundo banco que o cliente mais transaciona é o banco BTG, porém, o mesmo possui o menor valor transacionado, o que indica que o cliente faz muitas transações de menor valor para esse banco.

Também foi possível verificar que há um alto índice de tentativas de fraude na conta desse cliente, sendo que todas as tentativas de fraude foram com valores acima de R$19.999,00 e com a categoria de transferência. Por isso, foi criado um algoritimo de machine learning que identifica esses tipos de transações que contém fraude.


Conclui-se que há uma alta tentativa de transações com fraude e uma ação possível seria diminuir o limite máximo de transferência de pix do cliente.Possivelmente o cliente esteja usando essa conta PF com propósitos de PJ, devido a alta taxa de transferências s altos valores.  




  